<a href="https://colab.research.google.com/github/kxk302/Covid_Clustering/blob/main/Covid_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!ls '/content/gdrive/MyDrive/Colab Notebooks/Clustering'

boston_data  boston_results  uk_data  uk_results


In [5]:
import ast
import bokeh.models as bmo
import numpy as np
import pandas as pd

from bokeh.palettes import d3
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from matplotlib import pyplot as plt
from scipy.stats import entropy
from scipy.stats import gaussian_kde
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn import metrics

pd.set_option('display.max_columns', None)

# x axis values for calculating/plotting KDE of sample AF
x_idx = np.linspace(0.00, 1.00, num=100).tolist()

def get_kde_values(row):
  return gaussian_kde(row['AF']).evaluate(x_idx).tolist()

def get_kl_div(x, y):
  return entropy(x, y)

# "https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/var/bos_by_sample.tsv.gz", sep="\t"
# eps: 0.0025 for 5 clusters in Boston dataset
# metric: get_kl_div

# eps: 
#   The maximum distance between two samples for one to be considered as in the neighborhood of the other. This is the most important DBSCAN parameter to choose appropriately for your data set and distance function.
# min_samples: 
#   The number of samples n a neighborhood for a point to be considered as a core point. This includes the point itself.
# metric: 
#   The metric to use when calculating distance between instances in a feature array. 
# metric_params: 
#  Additional keyword arguments for the metric function.

def dbscan_clustering(file_name, sep='\t', eps=0.5, min_samples=5, metric='euclidean', metric_params=None):
  # Read the input file. Select only the needed columns.
  df_in = pd.read_csv(file_name, sep)[['Sample' , 'AF']]

  # Sample stats
  print('\n')
  print('Number of unique samples: {}'.format(df_in['Sample'].nunique()))
  print('Sample minimum: {}'.format(df_in['Sample'].min()))
  print('Sample maximum: {}'.format(df_in['Sample'].max()))

  # af stats
  print('\n')
  print('Number of unique af {}'.format(df_in['AF'].nunique()))
  print('af minimum: {}'.format(df_in['AF'].min()))
  print('af maximum: {}'.format(df_in['AF'].max()))

  # Clean up data by removing rows where af is greater than 1.0
  print('\n')
  print('Removing rows with AF greater than 1.0')
  df = df_in[df_in.AF <= 1.00]

  # Pivot the data frame
  df_piv = pd.pivot_table(df, index='Sample', values='AF', aggfunc=list)
  print('df_piv.head(5)')
  print(df_piv.head(5))

  # Clean up data by removing rows where af list has only one or two element
  # KDE calculation errors out for those
  df_piv_clean = df_piv[ df_piv.AF.str.len() > 2]

  # Calculate 
  df_piv_clean['KDE_vals'] = df_piv_clean.apply(get_kde_values, axis=1)

  print('df_piv_clean.head(5)')
  print(df_piv_clean.head(5))

  # Run DBSCAN clustering algorithm
  db = DBSCAN(eps=eps, min_samples=min_samples, metric=metric, metric_params=metric_params).fit(df_piv_clean.KDE_vals.tolist())
  labels = db.labels_

  # Number of clusters in labels, ignoring noise if present.
  n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
  n_noise_ = list(labels).count(-1)

  print('\n')
  print('Number of clusters: {}'.format(n_clusters_))
  print('Cluster labels: {}'.format(set(labels)))
  print('Number of noise samples: {}'.format(n_noise_))

  # Add Labels (and its string version) to the dataframe
  df_piv_clean['Labels'] = labels

  print('df_piv_clean.head(5)')
  print(df_piv_clean.head(5))

  return df_piv_clean

def get_distance_matrix(df_in):
  if df_in is None or df_in.shape[0] == 0:
    return df_in

  df = df_in.copy()

  row_count = df.shape[0]
  distances = np.zeros((row_count, row_count))

  for idx1 in range(row_count-1):
    for idx2 in range(idx1+1, row_count):
      distances[idx1][idx2] = entropy(df.iloc[idx1]['KDE_vals'], df.iloc[idx2]['KDE_vals'])
      distances[idx2][idx1] = distances[idx1][idx2]
  
  df_out = pd.DataFrame(distances)
  distances_sum = df_out.apply(np.sum)
  argmin = distances_sum.argmin()
  return df_out, df.iloc[argmin]

def plot_clusters(df_in, folder):
  if df_in is None or df_in.shape[0] == 0:
    return df_in

  df = df_in.copy()

  num_labels = df['Labels'].nunique()
  print('num_labels: {}'.format(num_labels))

  labels = df['Labels'].unique()
  print('labels: {}'.format(labels))

  fig, axs = plt.subplots(num_labels, 2, gridspec_kw={'hspace': 1.0, 'wspace': 0.5}, figsize=(15, 15))

  # Use num_labels - 1 in range, as we handle noise (-1) separately
  for label in labels:
    print('Label processed: {}'.format(label))

    # idx used in plot axes
    idx = 0
    if label != -1:
      idx = label
    else:
      idx = num_labels - 1

    df_lbl = df[ df.Labels == label ]
    
    distances, cluster_center = get_distance_matrix(df_lbl)
    print('Cluster center for label ' + str(label))
    print(cluster_center)
    
    # Histogram
    xh = cluster_center[0]
    axs[idx][0].hist(xh, density=True)
    axs[idx][0].title.set_text('Cluster ' + str(label) + ' (size ' + str(df_lbl.shape[0]) + ') AF histogram')

    # KDE 
    xk = x_idx
    yk = cluster_center[1]
    axs[idx][1].plot(xk, yk)
    axs[idx][1].title.set_text('Cluster ' + str(label) + ' (size ' + str(df_lbl.shape[0]) + ') AF density estimate')
    
  plt.savefig(folder + '/dbscan_' + str(num_labels) + '.png')
  plt.show()
  
def get_cluster_positions(df_in, cluster_label, file_name, sep):
  if df_in is None or df_in.shape[0] == 0:
    return df_in
  
  df = df_in.copy()
  df = df[ df.Labels == cluster_label ]

  # Read the input file. Select only the needed columns.
  df_positions = pd.read_csv(file_name, sep)[['Sample' , 'POS']]
  return df_positions[ df_positions.Sample.isin(df.index.tolist()) ]




In [ ]:
# Run DBSCAN clustering algorithm on Boston Covid-19 dataset
#
# Using 0.0025 for esp (epsilon) to yield 5 clusters
# Using default value of 5 for min_samples 
# Using get_kl_div method for metric. get_kl_div() calculates Kullback-Leibler divergence, which measures the distance between 2 proabaility distributions
# Using None for metric_params, as the metric has no parameters
#
df = dbscan_clustering(file_name='https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/var/bos_by_sample.tsv.gz', 
                       sep='\t', 
                       eps=0.0025, 
                       min_samples=5, 
                       metric=get_kl_div,
                       metric_params=None)

plot_clusters(d, folder='/content/gdrive/MyDrive/Colab Notebooks/Clustering/boston_results')

labels = df['Labels'].unique()
print('labels: {}'.format(labels))

# Use num_labels - 1 in range, as we handle noise (-1) separately
for label in labels:
  print('Label processed: {}'.format(label))

  # Get positions for cluster 3 samples 
  cluster_df = get_cluster_positions(df_in=df, cluster_label=label, file_name='https://github.com/galaxyproject/SARS-CoV-2/raw/master/data/var/bos_by_sample.tsv.gz', sep='\t')
  cluster_df.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Clustering/boston_results/cluster_' + str(label) + '.csv', sep=',', index=False)



In [ ]:
# Run DBSCAN clustering algorithm on UK Covid-19 dataset
#
# Using 0.0025 for esp (epsilon) to yield 5 clusters
# Using default value of 5 for min_samples 
# Using get_kl_div method for metric. get_kl_div() calculates Kullback-Leibler divergence, which measures the distance between 2 proabaility distributions
# Using None for metric_params, as the metric has no parameters
#
df = dbscan_clustering(file_name='/content/gdrive/MyDrive/Colab Notebooks/Clustering/uk_data/data_10_per.tsv', 
                       sep='\t', 
                       eps=0.0025, 
                       min_samples=5, 
                       metric=get_kl_div,
                       metric_params=None)

plot_clusters(df, folder='/content/gdrive/MyDrive/Colab Notebooks/Clustering/uk_results')

labels = df['Labels'].unique()
print('labels: {}'.format(labels))

# Use num_labels - 1 in range, as we handle noise (-1) separately
for label in labels:
  print('Label processed: {}'.format(label))

  # Get positions for cluster 3 samples 
  cluster_df = get_cluster_positions(df_in=df, cluster_label=label, file_name='/content/gdrive/MyDrive/Colab Notebooks/Clustering/uk_data/data_10_per.tsv', sep='\t')
  cluster_df.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Clustering/uk_results/cluster_' + str(label) + '.tsv', sep='\t', index=False)



Number of unique samples: 19855
Sample minimum: ERR5388794
Sample maximum: ERR5694900


Number of unique af 8400
af minimum: 0.05
af maximum: 594.6389


Removing rows with AF greater than 1.0
df_piv.head(5)
                                                           AF
Sample                                                       
ERR5388794  [0.8444, 0.8944, 0.8112, 0.9633, 0.972, 0.9751...
ERR5388795  [0.8929, 0.0552, 0.8959999999999999, 0.7854, 0...
ERR5388796  [0.8596, 0.8929, 0.7951, 0.9667, 0.9501, 0.965...
ERR5388797  [0.8533, 0.9074, 0.723, 0.8108, 0.9675, 0.9729...
ERR5388798  [0.8789, 0.8966, 0.795, 0.9690000000000001, 0....
df_piv_clean.head(5)
                                                           AF  \
Sample                                                          
ERR5388794  [0.8444, 0.8944, 0.8112, 0.9633, 0.972, 0.9751...   
ERR5388795  [0.8929, 0.0552, 0.8959999999999999, 0.7854, 0...   
ERR5388796  [0.8596, 0.8929, 0.7951, 0.9667, 0.9501, 0.965...   
ERR5388797 